In [1]:
import numpy as np

n_time = 1000
n_position_bins = 2000

log_likelihood = np.zeros((n_time, n_position_bins))


In [2]:
n_spikes = 4
joint_mark_intensity = 1 + np.arange(n_spikes)[:, np.newaxis] * np.ones(
    (1, n_position_bins)
)
joint_mark_intensity


array([[1., 1., 1., ..., 1., 1., 1.],
       [2., 2., 2., ..., 2., 2., 2.],
       [3., 3., 3., ..., 3., 3., 3.],
       [4., 4., 4., ..., 4., 4., 4.]])

In [3]:
spike_ind = [1, 5, 5, 6]


In [4]:
true_log_likelihood = np.zeros((n_time, n_position_bins))
true_log_likelihood[1] = 1
true_log_likelihood[5] = 5
true_log_likelihood[6] = 4
true_log_likelihood


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Option 1 - iterate over position bins, use bin count and stack

In [5]:
log_likelihood += np.stack(
    [
        np.bincount(spike_ind, weights=jmi, minlength=log_likelihood.shape[0])
        for jmi in joint_mark_intensity.T
    ],
    axis=1,
)


In [6]:
%%timeit

np.stack([np.bincount(spike_ind, weights=jmi, minlength=log_likelihood.shape[0]) for jmi in joint_mark_intensity.T], axis=1)

9.58 ms ± 263 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
assert np.allclose(log_likelihood, true_log_likelihood)


### Option 2 - use magical csr matrix incantation

In [8]:
log_likelihood = np.zeros((n_time, n_position_bins))


In [9]:
from scipy.sparse import csr_matrix

# https://stackoverflow.com/questions/5205345/using-numpy-bincount-with-array-weights

log_likelihood[: np.max(spike_ind) + 1] += (
    csr_matrix((np.ones((n_spikes,)), (spike_ind, np.arange(n_spikes))))
    * joint_mark_intensity
)


In [10]:
%%timeit

csr_matrix((np.ones((n_spikes,)), (spike_ind, np.arange(n_spikes)))) * joint_mark_intensity

In [ ]:
assert np.allclose(log_likelihood, true_log_likelihood)
